<a href="https://colab.research.google.com/github/p0lsol/programming_basics_ML/blob/main/ML_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re

with open('The-Books-of-Earthsea.txt') as f:
 text = f.read()
dataset = re.sub(r'\s+', ' ', re.sub(r'[^\w\s.]', '', text.lower())).strip()
dataset = dataset.split('.')

In [2]:
data = dataset[:5000] #попробовала 10000, было ту мач для колаба...

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

In [4]:

# Инициализируем токенизатор
tokenizer = Tokenizer()

# Обучаем токенизатор на заголовках
tokenizer.fit_on_texts(data)

# Преобразуем заголовки в последовательности чисел
sequences = tokenizer.texts_to_sequences(data)

# Создаем входные и выходные данные
X = []
y = []
for seq in sequences:
    for i in range(1, len(seq)):
        X.append(seq[:i])
        y.append(seq[i])

In [5]:
X[:10], y[:10]

([[556],
  [556, 2158],
  [556, 2158, 1255],
  [556, 2158, 1255, 7],
  [556, 2158, 1255, 7, 1],
  [556, 2158, 1255, 7, 1, 578],
  [556, 2158, 1255, 7, 1, 578, 1],
  [556, 2158, 1255, 7, 1, 578, 1, 239],
  [556, 2158, 1255, 7, 1, 578, 1, 239, 3],
  [556, 2158, 1255, 7, 1, 578, 1, 239, 3, 219]],
 [2158, 1255, 7, 1, 578, 1, 239, 3, 219, 5])

In [6]:
# Преобразуем списки в массивы numpy
X = np.asarray(X, dtype="object")
y = np.array(y)

# Дополняем последовательности до одинаковой длины
X = pad_sequences(X)

# Преобразуем y в one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=len(tokenizer.word_index) + 1)

In [7]:
# Создаем модель
model = Sequential()

# Добавляем слой Embedding
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=X.shape[1]))

# Добавляем слой LSTM
model.add(LSTM(150, return_sequences=False))

# Добавляем полносвязный слой
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))

# Компилируем модель
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Выводим информацию о модели
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Обучаем модель
history = model.fit(X, y, epochs=50, batch_size=64, validation_split=0.2)

Epoch 1/50
1132/1132 ━━━━━━━━━━━━━━━━━━━━ 23s 16ms/step - accuracy: 0.0747 - loss: 6.7642 - val_accuracy: 0.1097 - val_loss: 6.2142
Epoch 2/50
1132/1132 ━━━━━━━━━━━━━━━━━━━━ 17s 15ms/step - accuracy: 0.1172 - loss: 5.8690 - val_accuracy: 0.1248 - val_loss: 5.9765
Epoch 3/50
1132/1132 ━━━━━━━━━━━━━━━━━━━━ 17s 15ms/step - accuracy: 0.1406 - loss: 5.4573 - val_accuracy: 0.1334 - val_loss: 5.9126
Epoch 4/50
1132/1132 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - accuracy: 0.1527 - loss: 5.1853 - val_accuracy: 0.1370 - val_loss: 5.9192
Epoch 5/50
1132/1132 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.1613 - loss: 4.9728 - val_accuracy: 0.1400 - val_loss: 5.9873
Epoch 6/50
1132/1132 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - accuracy: 0.1766 - loss: 4.7390 - val_accuracy: 0.1430 - val_loss: 6.0242
Epoch 7/50
1132/1132 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - accuracy: 0.1809 - loss: 4.5618 - val_accuracy: 0.1436 - val_loss: 6.0959
Epoch 8/50
1132/1132 ━━━━━━━━━━━━━━━━━━━━ 17s 15ms/step - accuracy: 0.1913 -

In [9]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 141, 100)            │         724,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 150)                 │         150,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 7248)                │       1,094,448 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,909,546 (22.54 MB)

 Trainable params: 1,969,848 (7.51 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,939,698 (15.03 MB)

In [10]:

# Функция для генерации текста
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Генерируем новый заголовок
generated_text = generate_text("Magic", 10, X.shape[1])
print(generated_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Magic he said is a man once when his own saying


In [11]:
generated_text = generate_text("Dragon", 10, X.shape[1])
print(generated_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Dragon spoke with that word for a while the priestkings came


In [12]:
generated_text = generate_text("Duny", 10, X.shape[1])
print(generated_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Duny was not and could to ask for a word to


In [14]:
model.save('earthsea.keras')